## Option 1: Seam Tokenization Prototype

### Loading all the .obj files

In [5]:
import trimesh
from glob import glob

# mapping from obj to mesh
meshes = {}
for path in glob("8samples/*"):
    mesh = trimesh.load(path)
    meshes[path.split('/')[1]] = mesh
meshes

{'branch.obj': <trimesh.Trimesh(vertices.shape=(2767, 3), faces.shape=(1960, 3))>,
 'cylinder.obj': <trimesh.Trimesh(vertices.shape=(192, 3), faces.shape=(124, 3))>,
 'explosive.obj': <trimesh.Trimesh(vertices.shape=(2812, 3), faces.shape=(2566, 3))>,
 'fence.obj': <trimesh.Trimesh(vertices.shape=(1088, 3), faces.shape=(684, 3))>,
 'girl.obj': <trimesh.Trimesh(vertices.shape=(8284, 3), faces.shape=(8475, 3))>,
 'person.obj': <trimesh.Trimesh(vertices.shape=(3103, 3), faces.shape=(2251, 3))>,
 'table.obj': <trimesh.Trimesh(vertices.shape=(3148, 3), faces.shape=(4100, 3))>,
 'talwar.obj': <trimesh.Trimesh(vertices.shape=(1668, 3), faces.shape=(1922, 3))>}

### Step: 1 Identifying seams

In [6]:
import numpy as np
from collections import defaultdict

def detect_seams_trimesh(faces, verts, uvs, tol=1e-6):

    # Step 1: group vertex indices by their 3D position
    # (Because duplicated vertices represent seams)
    geom_groups = defaultdict(list)
    for vidx, vpos in enumerate(verts):
        geom_groups[tuple(np.round(vpos, 6))].append(vidx)

    # Step 2: among vertices with same 3D position, find which have different UVs
    seam_vertices = set()
    for group in geom_groups.values():
        if len(group) <= 1:
            continue

        # compare UVs inside the group
        for i in range(len(group)):
            for j in range(i+1, len(group)):
                if not np.allclose(uvs[group[i]], uvs[group[j]], atol=tol):
                    seam_vertices.add(group[i])
                    seam_vertices.add(group[j])

    # Step 3: edges connected to these vertices are seam edges
    seam_edges = set()

    for f in faces:
        edges = [(f[0], f[1]), (f[1], f[2]), (f[2], f[0])]
        for a, b in edges:
            if a in seam_vertices and b in seam_vertices:
                seam_edges.add(tuple(sorted((int(a), int(b)))))

    return sorted(list(seam_vertices)), sorted(list(seam_edges))

mesh = meshes['cylinder.obj']
faces = mesh.faces
verts = mesh.vertices
uvs = mesh.visual.uv

seam_vertices, seam_edges = detect_seams_trimesh(faces, verts, uvs)

print("Seam vertices:", seam_vertices)
print("Seam edges:", seam_edges)


Seam vertices: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191]
Seam edges: [(0, 3), (0, 7), (0, 9), (1, 5), (1, 187), (2, 8), (2, 188), (3, 9), (4, 10), (4, 16), (4, 185), (4, 189), (5, 187), (5, 1

In [7]:
import plotly.graph_objects as go

# --- Plot using Plotly ---
fig = go.Figure()

# Mesh
fig.add_trace(go.Mesh3d(
    x=verts[:,0], y=verts[:,1], z=verts[:,2],
    i=faces[:,0], j=faces[:,1], k=faces[:,2],
    color='lightblue', opacity=0.5, name='Mesh'
))

# Seams as red lines
for e in seam_edges:
    x = verts[e, 0]
    y = verts[e, 1]
    z = verts[e, 2]
    fig.add_trace(go.Scatter3d(
        x=x, y=y, z=z,
        mode='lines',
        line=dict(color='red', width=5),
        name='Seam'
    ))

fig.update_layout(
    scene=dict(aspectmode='data'),
    title="Mesh with Seams Highlighted"
)

fig.show()

In [17]:
def unit_sphere_normalize(vertices):
    centroid = vertices.mean(axis=0)
    centered = vertices - centroid
    scale = np.linalg.norm(centered, axis=1).max()
    return centered / scale, centroid, scale

def quantize(vertices, bins=1024):
    quantized = np.rint(vertices * (bins - 1)).astype(np.int32)
    return quantized

def quantize_unit_sphere(vertices, bins=1024):
    scaled = (vertices + 1) / 2     # [-1,1] -> [0,1]
    return quantize(scaled, bins)

def reconstruct_unitsphere(q_vertices, centroid, max_dist, bins=1024):
    scaled = q_vertices / (bins - 1)
    norm = scaled * 2 - 1
    return np.round(norm * max_dist + centroid,6)

In [9]:
def encode_seams(seam_edges, qverts):
    tokens = []
    for a, b in seam_edges:
        x1, y1, z1 = qverts[a]
        x2, y2, z2 = qverts[b]

        tokens.extend([
            "<EDGE>",
            int(x1), int(y1), int(z1),
            int(x2), int(y2), int(z2),
            "</EDGE>"
        ])
    return tokens

In [25]:
# 1. normalize
norm_verts, centroid, scale = unit_sphere_normalize(verts)

# 2. quantize
qverts = quantize_unit_sphere(norm_verts, bins=1024)

# 3. encode seams
tokens = encode_seams(seam_edges[:10], qverts)

without_encoding = []
for a, b in seam_edges:
    x1, y1, z1 = verts[a]
    x2, y2, z2 = verts[b]
    without_encoding.extend([(float(x1),float(y1),float(z1),float(x2),float(y2),float(z2))])

print("Original: ",without_encoding[:3])
print("Encoded: " ,tokens[:24])

Original:  [(0.0, -1.0, -1.0, 0.0, 1.0, -1.0), (0.0, -1.0, -1.0, 0.19509, -1.0, -0.980785), (0.0, -1.0, -1.0, 0.19509, 1.0, -0.980785)]
Encoded:  ['<EDGE>', 512, 150, 150, 512, 873, 150, '</EDGE>', '<EDGE>', 512, 150, 150, 582, 150, 157, '</EDGE>', '<EDGE>', 512, 150, 150, 582, 873, 157, '</EDGE>']


Current vocabulary:
- 2 special tokens: <EDGE>, </EDGE>
- 1024 coordinate tokens (0–1023)
- So total = 1026 tokens.

In [24]:
# recreate quantized seam vertices
decoded = []
for i in range(0, len(tokens), 8):
    if tokens[i] != "<EDGE>": 
        continue
    x1,y1,z1,x2,y2,z2 = tokens[i+1:i+7]
    decoded.append([(x1,y1,z1),(x2,y2,z2)])

# dequantize & denormalize
final_lines = []
for (qA, qB) in decoded:
    A = reconstruct_unitsphere(np.array(qA), centroid, scale)
    B = reconstruct_unitsphere(np.array(qB), centroid, scale)
    final_lines.append(tuple(map(float, A)) + tuple(map(float, B)))

print("Original:", without_encoding[:3])
print("Decoded: ",final_lines[:3])

Original: [(0.0, -1.0, -1.0, 0.0, 1.0, -1.0), (0.0, -1.0, -1.0, 0.19509, -1.0, -0.980785), (0.0, -1.0, -1.0, 0.19509, 1.0, -0.980785)]
Decoded:  [(0.001382, -0.999488, -0.999488, 0.001382, 0.999488, -0.999488), (0.001382, -0.999488, -0.999488, 0.194921, -0.999488, -0.980134), (0.001382, -0.999488, -0.999488, 0.194921, 0.999488, -0.980134)]
